In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import pandas as pd
DATAPATH = './data/'
gt = np.load(os.path.join(DATAPATH,"gt_1.npy"))
imu = np.loadtxt(os.path.join(DATAPATH,"seq_1.txt"))
wifi = pd.read_pickle(os.path.join(DATAPATH,"seq_1.pickle"))

C:\Users\ericw\miniconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import math
def distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return math.hypot(x1 - x2, y1 - y2)

def angle_between(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return math.atan2(y2 - y1, x2 - x1)

def normalize_angle(a):
    while a > math.pi:
        a -= 2*math.pi

    while a <= -math.pi:
        a += 2*math.pi
    
    return a
def extract_control(xs, ys, ts):
    # TODO: option to skip some indices?
    angle = angle_between((xs[0], ys[0]), (xs[1], ys[1]))
    # TODO: it is horrible to save the first pose here
    controls = [(xs[0], ys[0], angle, ts[0])]

    for i in range(len(xs) - 1):
        x, y, t = xs[i], ys[i], ts[i]
        next_x, next_y, next_t = xs[i+1], ys[i+1], ts[i+1]
        next_angle = angle_between((x, y), (next_x, next_y))
        angle_diff = normalize_angle(next_angle - angle)
        angle = next_angle
        d = distance((x, y), (next_x, next_y))
        ctrl = d, 0, angle_diff, next_t
        controls.append(ctrl)

    return controls

In [6]:
imu = np.loadtxt(os.path.join(DATAPATH,"seq_1.txt"))
imu

array([[ 5.10612760e+07,  2.59490479e-02,  2.69904546e-02,
        -4.84377705e-03, -4.52863760e-02],
       [ 5.10613760e+07,  7.25441054e-02,  6.43284917e-02,
        -2.65372097e-02, -2.58335680e-01],
       [ 5.10614760e+07,  2.24948153e-02,  2.44364776e-02,
         3.27952020e-03, -4.82229181e-02],
       ...,
       [ 5.11265770e+07,  2.36308072e-02,  2.47978121e-02,
         6.74758665e-03, -4.02833112e-02],
       [ 5.11266770e+07,  2.36061215e-02,  2.47854050e-02,
         6.84882700e-03, -4.03433032e-02],
       [ 5.11267770e+07,  2.36321054e-02,  2.47986279e-02,
         6.73927739e-03, -4.02981825e-02]])

In [212]:
DATAPATH = './data/'
gt = np.load(os.path.join(DATAPATH,"gt_1.npy"))
imu = np.loadtxt(os.path.join(DATAPATH,"seq_1.txt"))
wifi = pd.read_pickle(os.path.join(DATAPATH,"seq_1.pickle"))
imu_x = [0]
imu_y = [0]
for i in imu:
	imu_x.append(imu_x[-1]+i[3])
	imu_y.append(imu_y[-1]+i[4])
controls = extract_control(xs=imu_x[1:],ys=imu_y[1:],ts=imu[:,0])
observations = []
observations_time = []
for i in wifi:
	observations.append(wifi.get(i))
	observations_time.append(i)
heading = 0
new_gt =[]
for i in range(1,len(gt)):
	dy = gt[i,2] - gt[i-1,2]
	dx = gt[i,1] - gt[i-1,1]
	heading = math.atan2(dy, dx)
	print(heading)
	new_gt.append(np.array([gt[i,0],gt[i,1],gt[i,2],heading]))
gt = np.array(new_gt)


-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957559615793
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027
-1.2037957556353027
-1.2037957556039462
-1.2037957556353027


In [2]:
%load_ext autoreload
%autoreload 2
from DPF import *
dpf = DPF()

In [242]:
dpf.train(controls,observations,observations_time,gt)

c:\honours_project\fusion\pytorch-dpf\DPF.py:214: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.particles = torch.tensor(particles).to(device).to(torch.float32).unsqueeze(0)
c:\honours_project\fusion\pytorch-dpf\DPF.py:215: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.particle_probs = torch.tensor(self.particle_probs).to(device).to(torch.float32)


new
tensor(149.7281, device='cuda:0', grad_fn=<MeanBackward0>)
new
tensor(15.9720, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(158.5839, device='cuda:0', grad_fn=<MeanBackward0>)
new
tensor(2.2039, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(17.3107, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(152.7705, device='cuda:0', grad_fn=<MeanBackward0>)
new
tensor(17.5758, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(10.8230, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(22.0768, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(160.2481, device='cuda:0', grad_fn=<MeanBackward0>)
new
tensor(9.6365, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(20.1601, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(12.1908, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(22.7298, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(165.5191, device='cuda:0', grad_fn=<MeanBackward0>)
new
new
tensor(28.5349, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(97.9205, device='cuda:0', grad_fn=

KeyboardInterrupt: 

In [14]:
for i in range(1,3):
    gt = np.load("./particles_data/dpf_gt_" + str(i) + ".npy")
    observations = np.load("./particles_data/dpf_observations_" + str(i) + ".npy")
    particles = np.load("./particles_data/dpf_particles_" + str(i) + ".npy")
    gt = torch.tensor(gt).cuda()
    prob = particles[1:,:,4]
    particles = particles[1:,:,:4]
    observations = observations[1:]
    gt = gt[1:]
    prob = torch.tensor(prob).cuda()
    observations = torch.tensor(observations).cuda()
    particles = torch.tensor(particles).cuda()
    dpf.train_particles_data(particles,observations,gt)

TypeError: train_particles_data() missing 1 required positional argument: 'gt'

In [9]:
particles = np.load("./particles_data/dpf_particles_" + str(i) + ".npy")


In [13]:
np.shape(particles)

(241, 1000, 5)

In [12]:
particles[0][0]

array([ 54.07046818, -13.1484465 ,   0.58259412,   0.93483732,
         1.        ])